In [134]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import sqlite3 as sql

In [135]:
dtCarga = datetime.today().strftime('%d/%m/%Y %H:%M')

# COLETA DE DADOS DO SITE DO IBGE

In [136]:
#COLETANDO DADOS DO IBGE
url = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php#RJ'

DadosIBGE = pd.DataFrame(pd.read_html(url, match="Municípios do Rio de Janeiro")[0])

#RENOMEAR COLUNAS DO DF
DadosIBGE = DadosIBGE.rename(columns={'Municípios do Rio de Janeiro':'nmMunic','Códigos':'codMunic'})

#ALTERANDO NOME DO INDEX
DadosIBGE.index.name = 'idMunic'

#ALTERANDO INDEX PRA COMEÇAR EM 1
DadosIBGE.index = DadosIBGE.index + 1

#ADICIONAR UM COLUNA DE DATA E HORA DE CARGA
DadosIBGE['dtCarga'] = dtCarga

#DadosIBGE

# CRIANDO O BANCO DE DADOS ODS

In [137]:
#Manipulando o sistema de arquivos
endereco = Path('C:\\Users\\claud\\OneDrive\\Claudio Bonel-DADOTECA\\Projetos Sociais\\Projeto Dado Humanizado\\Pascoa com Dados\\')

BDODS = endereco / "pascoaODS.db"
BDDW = endereco / "pascoaDW.db"

if endereco.exists():
    if (BDODS.exists() and BDDW.exists()):
        print('Bancos de dados já existem!')
    else:
        BDODS.touch()
        BDDW.touch()
        print('Bancos de dados criados!')
else:
    print('Endereço não existe! Favor, verificar!')
        

Bancos de dados já existem!


# MANIPULANDO OS BANCOS DE DADOS CRIADOS

In [138]:
#Conectar no BDODS
conexaoODS = sql.connect(BDODS)

#Criar a tabela tbLogMunic e carregar os dados do DF DadosIBGE
DadosIBGE.to_sql('tbLogMunic',conexaoODS,if_exists="append")

#Confirmar a transação
conexaoODS.commit()

#Fechar a conexão
conexaoODS.close()

print('Carga do BDODS concluída!')

Carga do BDODS concluída!


In [139]:
#Conectar no BDDW
conexaoDW = sql.connect(BDDW)

#Selecionar somente as colunas para criação da dMunicipio
DadosIBGE = DadosIBGE[['nmMunic','codMunic']]

#Criar a tabela dMunicipio e carregar os dados do DF DadosIBGE
DadosIBGE.to_sql('dMunicipio',conexaoDW,if_exists="replace")

#Confirmar a transação
conexaoDW.commit()

#Fechar a conexão
conexaoDW.close()

print('Carga do DW concluída!')

Carga do DW concluída!


# CARGA DOS DADOS DE CADASTRO DAS DELEGACIAS DE POLÍCIA

In [140]:
#Definir o endereço físico do arquivo CSV
LocalArquivo = "C:\\Users\\claud\\OneDrive\\Claudio Bonel-DADOTECA\\CEPERJ\\Aulas\\Dados\\DADOS_POS\\"

#Criar Data Frame com os dados das DPs
tbLogDP = pd.DataFrame(pd.read_csv(LocalArquivo + "DP.csv"))

#Criando coluna com a data da carga
tbLogDP['dtCarga'] = dtCarga

#selecionando a coluna COD_DP como novo index
#tbLogDP = tbLogDP.set_index("COD_DP")

# CARGA DOS DADOS DE CADASTRO DOS RESPONSÁVEIS PELAS DELEGACIAS DE POLÍCIA

In [141]:
#Criar Data Frame com os dados dos responsáveis pelas DPs
tbLogRespDP = pd.DataFrame(pd.read_csv(LocalArquivo + "ResponsavelDP.csv"))

#Criando coluna com a data da carga
tbLogRespDP['dtCarga'] = dtCarga

#selecionando a coluna COD_DP como novo index
#tbLogRespDP = tbLogRespDP.set_index("COD_DP")

# CONECTANDO NO ODS E INFORMANDO AO PYTHON QUE USAREI LINGUAGEM SQL

In [142]:
#Conectar no BDODS
conexaoODS = sql.connect(BDODS)

#Definição de manipulação de dados em SQL
SQL_ODS = conexaoODS.cursor()

# INSERINDO DADOS DAS DELEGACIAS DE POLÍCIA, NO SQLITE (ODS) 

In [143]:
#Inserindo registros na tabela tbLogDP
print("Iniciando a inserção de dados na tabela tbLogDP.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogDP (codDP,nmDP,endereco,dtCarga) VALUES (?,?,?,?)''',tbLogDP.values.tolist())

#confirmar a transação
conexaoODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogDP), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela tbLogDP.
-------------------------------------------------
Carga finalizada! 137 registros inseridos na tbLogDP!


# INSERINDO DADOS DOS RESPONSÁVEIS PELAS DELEGACIAS DE POLÍCIA, NO SQLITE (ODS) 

In [144]:
#Inserindo registros na tabela tbLogDP
print("Iniciando a inserção de dados na tabela tbLogRespDP.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogRespDP (codDP,nmResponsavel,dtCarga) VALUES (?,?,?)''',tbLogRespDP.values.tolist())

#confirmar a transação
conexaoODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogRespDP), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela tbLogRespDP.
-------------------------------------------------
Carga finalizada! 137 registros inseridos na tbLogDP!


# VALIDANDO CARGA DOS DADOS DE DP E DE RESPONSÁVEL DP

In [145]:
#validar a carga através de um select na tblogDP
#pd.read_sql("SELECT * FROM tbLogDP",conexaoODS)

In [146]:
#validar a carga através de um select na tblogRespDP
#pd.read_sql("SELECT * FROM tbLogRespDP",conexaoODS)

# CRIANDO O DATAFRAME COM O SELECT DE CRIAÇÃO DA DIMENSÃO DP (dDP), NO DW

In [147]:
#Definir a query de criação da dimensão dDP

qry_dDP = '''
    SELECT 
        codDP,
        nmDP,
        endereco,
        nmResponsavel
    FROM
    (
        SELECT
            a.codDP,
            a.nmDP,
            a.endereco,
            b.nmResponsavel,
            max(a.dtCarga)
        FROM tbLogDP a
        JOIN tbLogRespDP b
        on a.codDP = b.codDP
        WHERE a.dtCarga = (SELECT max(x.dtCarga) FROM tbLogDP x)
        GROUP BY
            a.codDP,
            a.nmDP,
            a.endereco,
            b.nmResponsavel
    ) a
'''

dDP = pd.read_sql(qry_dDP,conexaoODS)

# COLETANDO DADOS DOS BATALHÕES DE POLÍCIA MILITAR (BPM)

In [148]:
#Criar Data Frame com os dados dos BPMs
tbLogBPM = pd.DataFrame(pd.read_excel(LocalArquivo + "BPM.xlsx"))

#Criando coluna com a data da carga
tbLogBPM['dtCarga'] = dtCarga


# CONSTRUINDO A TABELA DE LOG DE REGISTROS DO BPM (tbLogBPM), no ODS

In [149]:
#Definir a query de criação da tabela
qry_tbLogBPM = '''
    
    CREATE TABLE IF NOT EXISTS tbLogBPM (
    
    codBPM INTEGER,
    nmBPM VARCHAR (7),
    enderecoBPM VARCHAR (200),
    dtCarga DATETIME

    )
'''

#executar a criação da tbLogBPM
SQL_ODS.execute(qry_tbLogBPM)

#definição da qry de criação de index
qry_idx_tbLogBPM_codBPM = "CREATE INDEX IF NOT EXISTS idx_tbLogBPM_codBPM ON tbLogBPM (codBPM)"

#executar a criação do index
SQL_ODS.execute(qry_idx_tbLogBPM_codBPM)

# INSERINDO DADOS DE BPM NA TABELA tbLogBPM

In [150]:
#Inserindo registros na tabela tbLogBPM
print("Iniciando a inserção de dados na tabela tbLogBPM.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogBPM (codBPM,nmBPM,enderecoBPM,dtCarga) VALUES (?,?,?,?)''',tbLogBPM.values.tolist())

#confirmar a transação
conexaoODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogBPM), "registros inseridos na tbLogBPM!")

Iniciando a inserção de dados na tabela tbLogBPM.
-------------------------------------------------
Carga finalizada! 41 registros inseridos na tbLogBPM!


# COLETANDO DADOS DA ÁREA DOS BATALHÕES DE POLÍCIA MILITAR (BPM)

In [151]:
#Criar Data Frame com os dados dos BPMs
tbLogAreaBPM = pd.DataFrame(pd.read_csv(LocalArquivo + "areaBPMv1.csv"))

#Criando coluna com a data da carga
tbLogAreaBPM['dtCarga'] = dtCarga


# CONSTRUINDO A TABELA DE LOG DE REGISTROS DA ÁREA DOS BPMs (tbLogAreaBPM), no ODS

In [152]:
#Definir a query de criação da tabela
qry_tbLogAreaBPM = '''
    
    CREATE TABLE IF NOT EXISTS tbLogAreaBPM (
    
    codBPM INTEGER,
    areaBPM REAL (5,2),
    dtCarga DATETIME

    )
'''

#executar a criação da tbLogAreaBPM
SQL_ODS.execute(qry_tbLogAreaBPM)

#definição da qry de criação de index
qry_idx_tbLogAreaBPM_codBPM = "CREATE INDEX IF NOT EXISTS idx_tbLogAreaBPM_codBPM ON tbLogAreaBPM (codBPM)"

#executar a criação do index
SQL_ODS.execute(qry_idx_tbLogAreaBPM_codBPM)

# INSERINDO DADOS DA ÁREA DOS BPMs NA TABELA tbLogAreaBPM

In [153]:
#Inserindo registros na tabela tbLogAreaBPM
print("Iniciando a inserção de dados na tabela tbLogAreaBPM.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogAreaBPM (codBPM,areaBPM,dtCarga) VALUES (?,?,?)''',tbLogAreaBPM.values.tolist())

#confirmar a transação
conexaoODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogAreaBPM), "registros inseridos na tbLogAreaBPM!")

Iniciando a inserção de dados na tabela tbLogAreaBPM.
-------------------------------------------------
Carga finalizada! 41 registros inseridos na tbLogAreaBPM!


# CRIANDO O DATAFRAME COM O SELECT DE CRIAÇÃO DA DIMENSÃO DP (dBPM), NO DW

In [154]:
#Definir a query de criação da dimensão dBPM

qry_dBPM = '''
    SELECT
        codBPM,
        nmBPM,
        enderecoBPM,
        areaBPM
    FROM 
    (
        SELECT 
            a.codBPM,
            a.nmBPM,
            a.enderecoBPM,
            b.areaBPM,
            max(a.dtCarga)
        FROM tbLogBPM a
        JOIN tbLogAreaBPM b
        ON a.codBPM = b.codBPM
        WHERE a.dtCarga = (
                            SELECT MAX(x.dtCarga)
                            FROM tbLogBPM x
                       )
        GROUP BY
            a.codBPM,
            a.nmBPM,
            a.enderecoBPM,
            b.areaBPM
    ) a
'''

dBPM = pd.read_sql(qry_dBPM,conexaoODS)

# COLETA DOS DADOS DOS REGISTROS DE OCORRÊNCIAS

In [155]:
#Criar Data Frame com os dados dos registros das ocorrências
tbLogOcorrencias = pd.DataFrame(pd.read_csv(LocalArquivo + "OcorrenciaV2.csv"))

#Criando coluna com a data da carga
tbLogOcorrencias['dtCarga'] = dtCarga

#tbLogOcorrencias

# CRIANDO A TABELA tbLogOcorrencias, no SQLite, via SQL

In [156]:
#definição da qry de criação da tbLogOcorrrencias

qry_create_tbLogOcorrencias = '''

    CREATE TABLE IF NOT EXISTS tbLogOcorrencias (
    
    codDP INTEGER,
    codBPM INTEGER,
    ano INTEGER,
    mes INTEGER,
    mes_ano CHAR(7),
    regiao INTEGER,
    codMunicIBGE INTEGER,
    ocorrencias VARCHAR(100),
    qtde INTEGER,
    dtCarga DATETIME
    
    )

'''

#criando a tbLogOcorrencias
SQL_ODS.execute(qry_create_tbLogOcorrencias)

#Criação de index
SQL_ODS.execute("CREATE INDEX IF NOT EXISTS idx_tbLogOcorrencias ON tbLogOcorrencias (codDP,codBPM,codMunicIBGE)")

#confirmar a transação
conexaoODS.commit()

# CARREGANDO DADOS NA tbLogOcorrencias


In [157]:
#inserindo dados na tbLogOcorrencias
print("Iniciando a carga de dados na tbLogOcorrencias")
print("-----------------------------------------------")
SQL_ODS.executemany('''

    INSERT INTO tbLogOcorrencias (codDP,codBPM,ano,mes,mes_ano,regiao,codMunicIBGE,ocorrencias,qtde,dtCarga)
    VALUES(?,?,?,?,?,?,?,?,?,?)    
    ''', 
    tbLogOcorrencias.values.tolist()

)

#confirmar a transação
conexaoODS.commit()

print("Fim da carga de dados da tbLogOcorrencias!",len(tbLogOcorrencias),"Registros inseridos!")

Iniciando a carga de dados na tbLogOcorrencias
-----------------------------------------------
Fim da carga de dados da tbLogOcorrencias! 124006 Registros inseridos!


# CRIANDO O DATAFRAME COM O RESULTADO DA QUERY DE CARGA DE DADOS DAS OCORRÊNCIAS

In [158]:
#definição da query de carga de dados

qry_fOcorrencias = '''

    SELECT
        a.codDP,
        a.codBPM,
        a.ano,
        a.mes,
        a.regiao,
        a.codMunicIBGE,
        a.ocorrencias,
        a.qtde
    FROM tbLogOcorrencias a
    WHERE a.dtCarga = (
                        SELECT MAX(dtCarga)
                        FROM tbLogOcorrencias
                        )

'''

#popular o dataframe
fOcorrencias = pd.read_sql(qry_fOcorrencias,conexaoODS)

fOcorrencias

,codDP,codBPM,ano,mes,regiao,codMunicIBGE,ocorrencias,qtde
0,1,5,2018,1,1,3304557,ameaca,7
1,1,5,2018,1,1,3304557,apreensao_drogas,3
2,1,5,2018,1,1,3304557,estelionato,81
3,1,5,2018,1,1,3304557,estupro,1
4,1,5,2018,1,1,3304557,extorsao,3
...,...,...,...,...,...,...,...,...
124001,168,33,2018,12,3,3304409,recuperacao_veiculos,1
124002,168,33,2018,12,3,3304409,roubo_rua,1
124003,168,33,2018,12,3,3304409,roubo_transeunte,1
124004,168,33,2018,12,3,3304409,tentat_hom,2


# FECHANDO A CONEXÃO COM O ODS

In [159]:
#Confirmar a transação
conexaoODS.commit()

#Fechar a conexão
conexaoODS.close()

# CONECTANDO NO DW E INFORMANDO AO PYTHON QUE USAREI LINGUAGEM SQL

In [160]:
#Conectar no BDDW
conexaoDW = sql.connect(BDDW)

#Definição de manipulação de dados em SQL
SQL_DW = conexaoDW.cursor()

# DELETANDO DADOS ANTIGOS E REINICIANDO AO AUTOINCREMENTAL DA CHAVE, DA TABELA dDP

In [161]:
#Deletar os dados atuais
SQL_DW.execute("DELETE FROM dDP")

#Reiniciando o autoincramental da chave
SQL_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dDP'")

#Confirmar a transação
conexaoDW.commit()

# INSERINDO DADOS NA DIMENSÃO DP (dDP), NO DW

In [162]:
#Inserindo registros na tabela dDP
print("Iniciando a inserção de dados na tabela dDP.")
print("-------------------------------------------------")

SQL_DW.executemany('''INSERT INTO dDP (codDP,nmDP,endereco,nmResponsavel) VALUES (?,?,?,?)''',dDP.values.tolist())

#confirmar a transação
conexaoDW.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dDP), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela dDP.
-------------------------------------------------
Carga finalizada! 137 registros inseridos na tbLogDP!


# VALIDANDO CARGA DOS DADOS DA DIMENSÃO DP (dDP)

In [163]:
#validar a carga através de um select na tblogDP
#pd.read_sql("SELECT * FROM dDP",conexaoDW)

# CONSTRUINDO A TABELA DE DIMENSÃO BPM (dBPM), no DW

In [164]:
#Definir a query de criação da tabela
qry_dBPM = '''
    
    CREATE TABLE IF NOT EXISTS dBPM (
    
    idBPM INTEGER PRIMARY KEY AUTOINCREMENT,
    codBPM INTEGER,
    nmBPM VARCHAR (7),
    enderecoBPM VARCHAR (200),
    areaBPM REAL (5,2)

    )
'''

#executar a criação da tbLogBPM
SQL_DW.execute(qry_dBPM)

#executar a criação do index idBPM
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_dBPM_idBPM ON dBPM (idBPM)")

#executar a criação do index codBPM
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_dBPM_codBPM ON dBPM (codBPM)")

#Confirmar a transação
conexaoDW.commit()

# CONSTRUINDO A TABELA DE DIMENSÃO PERIODO (dPeriodo), no DW

In [165]:
#Definir a query de criação da tabela
qry_dPeriodo = '''
    
    CREATE TABLE IF NOT EXISTS dPeriodo (
    
    idPeriodo INTEGER PRIMARY KEY AUTOINCREMENT,
    data DATETIME,
    mes INTEGER,
    ano INTEGER,
    trimestre INTEGER,
    semestre INTEGER

    )
'''

#executar a criação da dPeriodo
SQL_DW.execute(qry_dPeriodo)

#executar a criação do index idPeriodo
SQL_DW.execute("CREATE UNIQUE INDEX IF NOT EXISTS idx_dPeriodo_idPeriodo ON dPeriodo (idPeriodo)")

#executar a criação do index codBPM
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_dPeriodo ON dPeriodo (mes,ano,trimestre,semestre)")

#Confirmar a transação
conexaoDW.commit()

# DELETANDO DADOS ANTIGOS E REINICIANDO AO AUTOINCREMENTAL DA CHAVE, DA TABELA dBPM

In [166]:
#Deletar os dados atuais
SQL_DW.execute("DELETE FROM dBPM")

#Reiniciando o autoincramental da chave
SQL_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dBPM'")

#Confirmar a transação
conexaoDW.commit()

# INSERINDO DADOS NA DIMENSÃO BPM (dBPM), NO DW

In [167]:
#Inserindo registros na tabela dBPM
print("Iniciando a inserção de dados na tabela dBPM.")
print("-------------------------------------------------")

SQL_DW.executemany('''INSERT INTO dBPM (codBPM,nmBPM,enderecoBPM,areaBPM) VALUES (?,?,?,?)''',dBPM.values.tolist())

#confirmar a transação
conexaoDW.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dBPM), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela dBPM.
-------------------------------------------------
Carga finalizada! 41 registros inseridos na tbLogDP!


# DELETANDO DADOS ANTIGOS E REINICIANDO AO AUTOINCREMENTAL DA CHAVE, DA TABELA dPeriodo

In [168]:
#Deletar os dados atuais
SQL_DW.execute("DELETE FROM dPeriodo")

#Reiniciando o autoincramental da chave
SQL_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dPeriodo'")

#Confirmar a transação
conexaoDW.commit()

# INSERINDO DADOS NA DIMENSÃO PERÍODO (dPeriodo), NO DW

In [169]:
#Inserindo registros na tabela dPeriodo
print("Iniciando a inserção de dados na tabela dPeriodo.")
print("-------------------------------------------------")

#definindo query de carga de dados da dPeriodo
qry_dados_dPeriodo = '''

    WITH data(d) AS
        (
            VALUES('2018-01-01')
            UNION ALL
            SELECT date(d,'+1 month')
            FROM data
            WHERE d < date('now')
         )
    SELECT 
         strftime('%d/%m/%Y', d) as data,
         cast(strftime('%m', d) as integer) as mes,
         cast(strftime('%Y', d) as integer) as ano,
         case
             when cast(strftime('%m', d) as integer) in (1,2,3) then 1 
             when cast(strftime('%m', d) as integer) in (4,5,6) then 2
             when cast(strftime('%m', d) as integer) in (7,8,9) then 3
                                                                else 4
         end as trimestre,
         case
             when cast(strftime('%m', d) as integer) in (1,2,3,4,5,6) then 1
                                                                      else 2
         end as semestre                                             
    FROM data;

'''
#Carregando dados no DF dPeriodo
dPeriodo = pd.read_sql(qry_dados_dPeriodo,conexaoDW)

#Executar a carga de dados na dPeriodo
SQL_DW.executemany('''INSERT INTO dPeriodo (data,mes,ano,trimestre,semestre) VALUES (?,?,?,?,?)''',dPeriodo.values.tolist())

#confirmar a transação
conexaoDW.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dPeriodo), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela dPeriodo.
-------------------------------------------------
Carga finalizada! 53 registros inseridos na tbLogDP!


# CRIANDO A TABELA FATO OCORRENCIAS, NO DW

In [170]:
#definindo a qry da criação da fOcorrencias

qry_create_fOcorrencias = '''

    CREATE TABLE IF NOT EXISTS fOcorrencias (
    
    idDP INTEGER REFERENCES dDP(idDP) ON UPDATE NO ACTION ON DELETE NO ACTION,
    idBPM INTEGER REFERENCES dBPM(idBPM) ON UPDATE NO ACTION ON DELETE NO ACTION,
    idPeriodo INTEGER REFERENCES dPeriodo(idPeriodo) ON UPDATE NO ACTION ON DELETE NO ACTION,
    regiao INTEGER,
    idMunic INTEGER REFERENCES dMunicipio(idMuni) ON UPDATE NO ACTION ON DELETE NO ACTION,
    ocorrencias VARCHAR(100),
    qtde INTEGER
    
    )

'''

#criação da fOcorrencias
SQL_DW.execute(qry_create_fOcorrencias)

#criação de index
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_fOcorrencias ON fOcorrencias(idDP,idBPM,idPeriodo,idMunic)")
               
#confirmar a transação
conexaoDW.commit()

# INSERINDO DADOS NA FATO OCORRÊNCIAS (fOcorrencias) - CARGA INCREMENTAL

In [171]:
#criar um tabela temporária
fOcorrencias.to_sql('tempOcorrencias',conexaoDW,if_exists="replace")

In [172]:
#definindo a query de carga incremental
qry_carga_incremental_focorrencias = '''

    SELECT
        b.idDP,
        c.idBPM,
        d.idPeriodo,
        a.regiao,
        e.idMunic,
        a.ocorrencias,
        a.qtde
    FROM tempOcorrencias a
    JOIN dDP b
    ON a.codDP = b.codDP
    JOIN dBPM c
    ON a.codBPM = c.codBPM
    JOIN dPeriodo d
    ON (a.ano = d.ano) AND (a.mes = d.mes)
    JOIN dMunicipio e
    ON a.codMunicIBGE = e.codMunic
    LEFT JOIN fOcorrencias g
    ON b.idDP = g.idDP
    AND c.idBPM = g.idBPM
    AND d.idPeriodo = g.idPeriodo
    AND e.idMunic = g.idMunic
    WHERE g.idDP is null
    AND g.idBPM is null
    AND g.idPeriodo is null
    AND g.idMunic is null

'''

#criando um DF Incremental
carga_fOcorrencias = pd.read_sql(qry_carga_incremental_focorrencias,conexaoDW)

carga_fOcorrencias

,idDP,idBPM,idPeriodo,regiao,idMunic,ocorrencias,qtde


In [173]:
#carregando dados na tabela fOcorrencias
print("Carregando dados na tabela fOcorrencias.")
print("----------------------------------------")

SQL_DW.executemany('''

    INSERT INTO fOcorrencias (idDP,idBPM,idPeriodo,regiao,idMunic,ocorrencias,qtde)
    VALUES(?,?,?,?,?,?,?)''',
    
    carga_fOcorrencias.values.tolist()
)



#confirmar a transação
conexaoDW.commit()

print("Fim da carga incremental de dados na fOcorrencias!",len(carga_fOcorrencias),"registros inseridos!")

Carregando dados na tabela fOcorrencias.
----------------------------------------
Fim da carga incremental de dados na fOcorrencias! 0 registros inseridos!


# ATUALIZANDO DADOS NA FATO OCORRÊNCIAS (fOcorrencias) - ATUALIZAÇÃO RETROATIVA


In [174]:
#definindo a qry de verificação da existÊncia de atualizações retroativas
qry_atualizacao_fOcorrencias = '''

    SELECT
        a.qtde,
        b.idDP,
        c.idBPM,
        d.idPeriodo,
        a.regiao,
        e.idMunic,
        a.ocorrencias
    FROM tempOcorrencias a
    JOIN dDP b
    ON a.codDP = b.codDP
    JOIN dBPM c
    ON a.codBPM = c.codBPM
    JOIN dPeriodo d
    ON (a.ano = d.ano) AND (a.mes = d.mes)
    JOIN dMunicipio e
    ON a.codMunicIBGE = e.codMunic
    LEFT JOIN fOcorrencias g
    ON b.idDP = g.idDP
    AND c.idBPM = g.idBPM
    AND d.idPeriodo = g.idPeriodo
    AND e.idMunic = g.idMunic
    AND a.regiao = g.regiao
    AND a.ocorrencias = g.ocorrencias
    WHERE a.qtde <> g.qtde
    
'''

#criando um DF de atualização
atualiza_fOcorrencias = pd.read_sql(qry_atualizacao_fOcorrencias,conexaoDW)

atualiza_fOcorrencias

,qtde,idDP,idBPM,idPeriodo,regiao,idMunic,ocorrencias
0,7,1,5,1,1,68,ameaca
1,3,1,5,1,1,68,apreensao_drogas


In [175]:
#atualizando os dados retroativos na fOcorrencias
print("Iniciando atualiazação na fato Ocorrências!")

#definindo a qry de atualização
qry_atualiza_fOcorrencias = '''

    UPDATE fOcorrencias
    SET qtde = ?
    WHERE idDP = ?
    AND idBPM = ?
    AND idPeriodo = ?
    AND regiao = ?
    AND idMunic = ?
    AND ocorrencias = ?

'''
#atualizando dados na fOcorrencias
SQL_DW.executemany(qry_atualiza_fOcorrencias,atualiza_fOcorrencias.values.tolist())

#confirmar a transação
conexaoDW.commit()

print("Fim da atualização de dados na fato Ocorrências!",len(atualiza_fOcorrencias),"registros atualizados!")


Iniciando atualiazação na fato Ocorrências!
Fim da atualização de dados na fato Ocorrências! 2 registros atualizados!


In [176]:
#deletar a tabela temporária
SQL_DW.execute("DROP TABLE tempOcorrencias")

#Confirmar a transação
conexaoDW.commit()

# FECHANDO A CONEXÃO COM O DW

In [177]:
#Fechar a conexão
conexaoDW.close()